# Hotspots

Hotspots California map: 
- Wildifre-wildifreSmokePM2.5, 
- Heat-wildfire,
- Heat-PM2.5, 
- Wildifre-Heat-SmokePM2.5P

In [55]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
df = pd.read_parquet("outputs/merged_heatday_coldday_wfday_polluted_smoke_polluted_rolling.parquet")

In [56]:
df = df.drop(columns = ['tmin','tmax','pm25','smoke_pm'])

### Find hotspots

Terminology:
- h heat
- w wildfire
- p polluted
- s smoke polluted

Example:
- hwp is a hotspot where heat wildfire and pollution are co-occurring 

- _hws is where heat OR wildfire OR smoke are occurring on the same day

- hws_2d is where heat wildfire and smoke are occurring within 24 hours window (2 days)

In [57]:
df['hw'] = df['wfday'] & df['heatday']
df['hp'] = df['heatday'] & df['polluted']
df['hwp'] = df['heatday'] & df['wfday'] & df['polluted']

In [58]:
df['_hws'] = df[['heatday','wfday','smoke_pm_non_zero']].any(axis=1)

In [59]:
# smoke_pm_non_zero taken as main

df['hs'] = df['heatday'] & df['smoke_pm_non_zero']
df['hws'] = df['heatday'] & df['wfday'] & df['smoke_pm_non_zero']
df['ws'] = df['wfday'] & df['smoke_pm_non_zero']

In [60]:
df['hs5'] = df['heatday'] & df['smoke_pm_gt_five']
df['hws5'] = df['heatday'] & df['wfday'] & df['smoke_pm_gt_five']
df['ws5'] = df['wfday'] & df['smoke_pm_gt_five']

In [61]:
df = df.drop(columns=['index', 'FIPS_1'])

In [78]:
df.to_parquet("outputs/hotspots_per_fips_rolling.parquet")

## Hotspots in time window of 2D

In [18]:
df = pd.read_parquet("outputs/hotspots_per_fips_rolling.parquet")

In [62]:
df.set_index('time', inplace=True)

In [63]:
def get_time_hotspot(x):
    x['heatday_2d'] = x['heatday'].rolling('2D', min_periods=1).max()
    x['smoke_pm_non_zero_2d'] = x['smoke_pm_non_zero'].rolling('2D', min_periods=1).max()
    x['wfday_2d'] = x['wfday'].rolling('2D', min_periods=1).max()
    return x

df['heatday_2d'] = False  # Initialize the 'new' column as False
df['smoke_pm_non_zero_2d'] = False  # Initialize the 'new' column as False
df['wfday_2d'] = False  # Initialize the 'new' column as False

df = df.groupby(by=['GEOID']).apply(get_time_hotspot)

/var/folders/b_/vlxgm9n10bz569rffzmcc5dc0000gn/T/ipykernel_33428/830991533.py:11: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby(by=['GEOID']).apply(get_time_hotspot)


In [64]:
df['heatday_2d'] = df['heatday_2d'].astype(int)
df['smoke_pm_non_zero_2d'] = df['smoke_pm_non_zero_2d'].astype(int)
df['wfday_2d'] = df['wfday_2d'].astype(int)

In [65]:
df['_hws_2d'] = df[['heatday_2d','wfday_2d','smoke_pm_non_zero_2d']].any(axis=1)
df['hs_2d'] = df['heatday_2d'] & df['smoke_pm_non_zero_2d']
df['hws_2d'] = df['heatday_2d'] & df['wfday_2d'] & df['smoke_pm_non_zero_2d']
df['ws_2d'] = df['wfday_2d'] & df['smoke_pm_non_zero_2d']

In [66]:
df.columns

Index(['rhum', 'wfday', 'GEOID', 'heatday', 'coldday', 'polluted',
       'smoke_pm_non_zero', 'smoke_pm_gt_five', 'hw', 'hp', 'hwp', '_hws',
       'hs', 'hws', 'ws', 'hs5', 'hws5', 'ws5', 'heatday_2d',
       'smoke_pm_non_zero_2d', 'wfday_2d', '_hws_2d', 'hs_2d', 'hws_2d',
       'ws_2d'],
      dtype='object')

In [67]:
df.to_parquet("outputs/hotspots_per_fips_rolling.parquet") 